In [56]:
import pandas as pd
import numpy as np
import random
import gc
from os import listdir
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, BatchNormalization, LSTM
from sklearn.preprocessing import MinMaxScaler

In [7]:
import copy
import pandas as pd
import numpy as np
import librosa
import seaborn as sb
import matplotlib.pyplot as plt
import itertools
import re
import random
import gc
import math
import os
from sklearn.utils import shuffle
from operator import itemgetter, attrgetter, methodcaller
from os import listdir
from os.path import isfile, join
from numpy import median, diff
from xgboost import XGBClassifier
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.externals import joblib
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import EnsembleVoteClassifier
np.random.seed(0)

### TODOS
- generate percent of single double notes etc with a nn

## Classes
- 0: one note
- 1: two notes
- 2: three or four notes
- 3: hold start
- 4: roll start
- 5: mine

In [8]:
songs_to_use_full = pd.read_csv('data/songs_to_use.csv').values
save_files = listdir('data')
songs_to_use = [song_data for song_data in songs_to_use_full if '{0}_misc.csv'.format(song_data[0]) in save_files]
songs = [SongFile(song_data[0]) for song_data in songs_to_use]
np.random.shuffle(songs)

In [2]:
def get_class_for_index_expanded(notes, index):
    if index < 0:
        return [0, 0, 0, 0, 0, 0]
    row = notes[index][0]
    (steps, holds, rolls, mines) = [row.count(char) for char in ['1', '2', '4', 'M']]
    if steps == 0 and mines == 0 and holds == 0 and rolls == 0:
        return [0, 0, 0, 0, 0, 0]
    steps += (holds + rolls)
    return [int(i) for i in [steps == 1, steps == 2, steps > 2, holds > 0, rolls > 0, mines > 0]]

def get_class_for_index(notes, index):
    classes_expanded = get_class_for_index_expanded(notes, index)
    return [i for i in range(6) if classes_expanded[i]]

In [3]:
steps_per_bar = 48
class SongFile:
    def __init__(self, key):
        misc = pd.read_csv('data/{0}_misc.csv'.format(key)).values
        self.bpm = misc[1][0]
        self.notes = pd.read_csv('data/{0}_notes.csv'.format(key), converters={'0': lambda x: str(x)}).values
        self.note_classes = [get_class_for_index_expanded(self.notes, i) for i in range(len(self.notes))]

In [4]:
beats_to_track = 48
note_types = ['1', 'M', '2', '4', '3']

def get_features_for_row(row):
    return [int(char == target) for target in note_types for char in row]

empty_row = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
def get_previous_notes(index, features):
    previous_notes = [features[i] for i in range(index, index + song_padding) if not np.array_equal(features[i], empty_row)]
    return [empty_row] * (8 - len(previous_notes)) + previous_notes[-8:]
    
song_padding = beats_to_track * 2
song_end_padding = beats_to_track * 2
important_indices = [1, 2, 3, 4, 8, 16, 20, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96]
important_indices_classes = [-96, -84, -72, -60, -48, -36, -24, -12, 0, 1, 2, 3, 4, 8, 16, 20, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96]
def get_features(index, features, note_classes):
    indices = [index + song_padding - i for i in important_indices]
    indices_classes = [index + song_padding - i for i in important_indices_classes]
    past_classes = np.array([note_classes[i] for i in indices_classes]).flatten()
    past_features = np.array([features[i] for i in indices]).flatten()
    previous_notes = np.array(get_previous_notes(index, features)).flatten()
    return np.concatenate((past_classes, past_features, previous_notes), axis = 0)

In [69]:
beats_to_track = 48
note_types = ['1', 'M', '2', '4', '3']

def get_features_for_row(row):
    return [int(char == target) for target in note_types for char in row]

feature_important_indices = [1, 2, 3, 4, 6, 8, 12, 16, 18, 20, 24, 30, 36, 48, 60, 72, 84, 96]
note_important_indices = [-48, -24, -12, -6, -3, 0, 3, 4, 6, 8, 12, 16, 18, 24, 36, 48, 72, 96]

prev_note_length = len(important_indices)
empty_row = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
def get_previous_notes(index, features):
    previous_notes = [features[index + song_padding - i] for i in range(1, 96) if not np.array_equal(features[i], empty_row)]
    return [empty_row] * (prev_note_length - len(previous_notes)) + previous_notes[-prev_note_length:]

song_padding = 96
song_end_padding = 48

def get_features(index, features, note_classes):
    past_notes = get_previous_notes(index, features)
    feature_indices = [index + song_padding - i for i in feature_important_indices]
    note_indices = [index + song_padding - i for i in note_important_indices]
    return [np.concatenate((past_notes[i], features[feature_indices[i]], note_classes[note_indices[i]]), axis = 0) for i in range(prev_note_length)]

In [44]:
def get_model_output_for_class(model_class, row):
    if model_class == 0 or model_class == 1 or model_class == 2:
        return [int(char == '1' or char == '2' or char == '4') for char in row]
    if model_class == 3:
        return [int(char == '2') for char in row]
    if model_class == 4:
        return [int(char == '4') for char in row]
    if model_class == 5:
        return [int(char == 'M') for char in row]

def get_hold_length(notes, note_row, note_column):
    i = 0
    while i < len(notes) - note_row:
        if notes[note_row + i][0][note_column] == '3':
            return i
        i += 1
    return False

In [70]:
def get_features_for_songs(songs):
    hold_X = []
    roll_X = []
    hold_y = []
    roll_y = []
    X = [[] for i in range(6)]
    y = [[] for i in range(6)]
    for song in songs:
        note_classes = np.concatenate((([[0, 0, 0, 0, 0, 0]] * song_padding), song.note_classes, ([[0, 0, 0, 0, 0, 0]] * song_end_padding)), axis = 0)
        notes = np.concatenate((([['0000']] * song_padding), song.notes), axis = 0)
        if abs(len(note_classes) - len(notes) > 250):
            print ('Lengths dont match for {0}'.format(key))
            print ('{0} vs {1}'.format(len(note_classes), len(notes)))
            continue
        length = min(len(note_classes) - song_padding - song_end_padding, len(notes) - song_padding)
        features = np.array([get_features_for_row(notes[i][0]) for i in range(0, length + song_padding)])
        for i in range(length):
            row = notes[i + song_padding][0]
            model_classes = get_class_for_index(notes, i + song_padding)
            for model_class in model_classes:
                X_row = get_features(i, features, note_classes)
                X[model_class].append(X_row)
                y[model_class].append(get_model_output_for_class(model_class, row))
                
                if model_class == 3:
                    for j in range(4):
                        if row[j] == '2':
                            length = get_hold_length(notes, i + song_padding, j)
                            if length:
                                hold_X.append(X_row)
                                hold_y.append(length)
                if model_class == 4:
                    for j in range(4):
                        if row[j] == '4':
                            length = get_hold_length(notes, i + song_padding, j)
                            if length:
                                roll_X.append(X_row)
                                roll_y.append(length)

    X = [np.array(X_for_class) for X_for_class in X]
    y = [np.array(y_for_class) for y_for_class in y]
    return X, y, np.array(hold_X), np.array(hold_y), np.array(roll_X), np.array(roll_y)

In [71]:
%%time
X_train_array, y_train_array, hold_X_train, hold_y_train, roll_X_train, roll_y_train = get_features_for_songs(songs[:174]) # total 217
X_test_array, y_test_array, hold_X_test, hold_y_test, roll_X_test, roll_y_test = get_features_for_songs(songs[174:])

CPU times: user 2min 25s, sys: 3.36 s, total: 2min 28s
Wall time: 2min 30s


In [72]:
hold_lengths = [3, 6, 9, 12, 18, 24, 36, 48]
def get_closest_hold_lengths(lengths):
    closest = [np.argmax([-abs(length - aprox) for aprox in hold_lengths]) for length in lengths]
    closest_one_hot = np.zeros((len(closest), len(hold_lengths)))
    closest_one_hot[np.arange(len(closest)), closest] = 1
    return np.array(closest_one_hot)

In [73]:
hold_y_train = get_closest_hold_lengths(hold_y_train)
roll_y_train = get_closest_hold_lengths(roll_y_train)
hold_y_test = get_closest_hold_lengths(hold_y_test)
roll_y_test = get_closest_hold_lengths(roll_y_test)

In [54]:
def build_model(num_classes):
    model = Sequential()

    model.add(LSTM(128, batch_input_shape=[64, 18, 46], stateful=True))
    model.add(BatchNormalization())
    model.add(Activation('softsign'))

    model.add(Dense(num_classes))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adagrad',
                  metrics=['accuracy'])

    return model

In [74]:
X_train = X_train_array[0]
y_train = y_train_array[0]
X_test = X_test_array[0]
y_test = y_test_array[0]

In [75]:
model = Sequential()

model.add(LSTM(128, batch_input_shape=[64, 18, 46], stateful=True))
model.add(BatchNormalization())
model.add(Activation('softsign'))

model.add(Dense(4))
model.add(BatchNormalization())
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adagrad',
              metrics=['accuracy'])

batch_size = 64
train_cutoff = int(len(X_train) / batch_size) * batch_size
test_cutoff = int(len(X_test) / batch_size) * batch_size
model.fit(X_train[:train_cutoff], y_train[:train_cutoff], nb_epoch=8, batch_size=batch_size, verbose=1, validation_data=(X_test[:test_cutoff], y_test[:test_cutoff]))
print (model.evaluate(X_test[:test_cutoff], y_test[:test_cutoff], batch_size=batch_size))

Train on 78784 samples, validate on 18432 samples
Epoch 1/8
78784/78784 [==============================] - 74s - loss: 1.3649 - acc: 0.3185 - val_loss: 1.3820 - val_acc: 0.3128
Epoch 2/8
78784/78784 [==============================] - 66s - loss: 1.3399 - acc: 0.3516 - val_loss: 1.3447 - val_acc: 0.3503
Epoch 3/8
78784/78784 [==============================] - 64s - loss: 1.3271 - acc: 0.3610 - val_loss: 1.3362 - val_acc: 0.3544
Epoch 4/8
78784/78784 [==============================] - 96s - loss: 1.3168 - acc: 0.3663 - val_loss: 1.3312 - val_acc: 0.3580
Epoch 5/8
78784/78784 [==============================] - 179s - loss: 1.3097 - acc: 0.3713 - val_loss: 1.3196 - val_acc: 0.3632
Epoch 6/8
78784/78784 [==============================] - 68s - loss: 1.3026 - acc: 0.3756 - val_loss: 1.3160 - val_acc: 0.3670
Epoch 7/8
78784/78784 [==============================] - 66s - loss: 1.2962 - acc: 0.3786 - val_loss: 1.3172 - val_acc: 0.3639
Epoch 8/8
18432/18432 [==============================] - 4s 

In [76]:
X_train_flat = np.array([np.array(row).flatten() for row in X_train])
X_test_flat = np.array([np.array(row).flatten() for row in X_test])

In [77]:
model = Sequential()

model.add(Dense(512, input_shape=(828,)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.4))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.4))

model.add(Dense(4))
model.add(BatchNormalization())
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adagrad',
              metrics=['accuracy'])

batch_size = 64
train_cutoff = int(len(X_train) / batch_size) * batch_size
test_cutoff = int(len(X_test) / batch_size) * batch_size
model.fit(X_train_flat[:train_cutoff], y_train[:train_cutoff], nb_epoch=8, batch_size=batch_size, verbose=1, validation_data=(X_test_flat[:test_cutoff], y_test[:test_cutoff]))
print (model.evaluate(X_test_flat[:test_cutoff], y_test[:test_cutoff], batch_size=batch_size))

Train on 78784 samples, validate on 18432 samples
Epoch 1/8
78784/78784 [==============================] - 21s - loss: 1.2886 - acc: 0.3589 - val_loss: 1.2464 - val_acc: 0.3748
Epoch 2/8
78784/78784 [==============================] - 21s - loss: 1.2283 - acc: 0.3983 - val_loss: 1.2309 - val_acc: 0.3855
Epoch 3/8
78784/78784 [==============================] - 21s - loss: 1.2082 - acc: 0.4094 - val_loss: 1.2267 - val_acc: 0.3873
Epoch 4/8
78784/78784 [==============================] - 22s - loss: 1.1960 - acc: 0.4208 - val_loss: 1.2214 - val_acc: 0.3895
Epoch 5/8
78784/78784 [==============================] - 22s - loss: 1.1836 - acc: 0.4309 - val_loss: 1.2201 - val_acc: 0.3928
Epoch 6/8
78784/78784 [==============================] - 21s - loss: 1.1734 - acc: 0.4387 - val_loss: 1.2193 - val_acc: 0.3940
Epoch 7/8
78784/78784 [==============================] - 22s - loss: 1.1668 - acc: 0.4429 - val_loss: 1.2186 - val_acc: 0.3943
Epoch 8/8
18368/18432 [============================>.] - ETA:

In [59]:
model = build_model(len(hold_lengths))
model.fit(hold_X_train, hold_y_train, nb_epoch=5, batch_size=64, verbose=1, validation_data=(hold_X_test, hold_y_test))
model.save('models/hold_length_model.h5')

Train on 6985 samples, validate on 1629 samples
Epoch 1/5
6985/6985 [==============================] - 4s - loss: 1.5991 - acc: 0.4777 - val_loss: 1.8146 - val_acc: 0.3806
Epoch 2/5
6985/6985 [==============================] - 5s - loss: 1.1324 - acc: 0.6759 - val_loss: 1.7726 - val_acc: 0.3677
Epoch 3/5
6985/6985 [==============================] - 5s - loss: 0.8831 - acc: 0.7801 - val_loss: 1.7566 - val_acc: 0.3800
Epoch 4/5
6985/6985 [==============================] - 4s - loss: 0.6957 - acc: 0.8624 - val_loss: 1.9463 - val_acc: 0.3143
Epoch 5/5
6985/6985 [==============================] - 5s - loss: 0.5773 - acc: 0.9138 - val_loss: 1.8850 - val_acc: 0.3554


In [60]:
model = build_model(len(hold_lengths))
model.fit(roll_X_train, roll_y_train, nb_epoch=5, batch_size=64, verbose=1, validation_data=(roll_X_test, roll_y_test))
model.save('models/roll_length_model.h5')

Train on 195 samples, validate on 67 samples
Epoch 1/5
195/195 [==============================] - 0s - loss: 1.8828 - acc: 0.3333 - val_loss: 3.0420 - val_acc: 0.1045
Epoch 2/5
195/195 [==============================] - 0s - loss: 0.7856 - acc: 0.8923 - val_loss: 2.1661 - val_acc: 0.0896
Epoch 3/5
195/195 [==============================] - 0s - loss: 0.6140 - acc: 0.9538 - val_loss: 2.1022 - val_acc: 0.0896
Epoch 4/5
195/195 [==============================] - 0s - loss: 0.5359 - acc: 0.9692 - val_loss: 2.2033 - val_acc: 0.0746
Epoch 5/5
195/195 [==============================] - 0s - loss: 0.5048 - acc: 0.9846 - val_loss: 2.1390 - val_acc: 0.0746


In [81]:
class_arrays = [
    ['1000', '0100', '0010', '0001'],
    ['1100', '1010', '1001', '0110', '0101', '0011'],
    ['1110', '1101', '1011', '0111', '1111'],
    ['1000', '0100', '0010', '0001', '2', '3', '4'],
    ['1000', '0100', '0010', '0001', '2', '3', '4'],
    ['1000', '0100', '0010', '0001', '2', '3', '4'],
]
class_maps = [dict((class_array[i], i) for i in range(len(class_array))) for class_array in class_arrays]
def get_class(class_map, y_row):
    as_string = ''.join(str(x) for x in y_row)
    pos_count = as_string.count('1')
    return class_map[str(pos_count)] if '2' in class_map and pos_count > 1 else class_map[as_string]

def get_y_not_one_hot(y):
    return [[get_class(class_map, y_row) for y_row in y_section] for class_map, y_section in zip(class_maps, y)]

In [82]:
y_train_classes = get_y_not_one_hot(y_train_array)
y_test_classes = get_y_not_one_hot(y_test_array)

# Note Models

In [79]:
%%time
min_samples_leafs = [16, 4, 1, 4, 1, 2]
clf = RandomForestClassifier(n_estimators = 50, min_samples_leaf=16, max_features=600)
clf.fit(X_train_flat, y_train)
clf.score(X_test_flat, y_test)

CPU times: user 5min 48s, sys: 3.33 s, total: 5min 51s
Wall time: 6min 10s


In [80]:
clf.score(X_test_flat, y_test)

0.10965506037797151

In [106]:
%%time
min_samples_leafs = [16, 4, 1, 4, 1, 2]
for X_train, y_train, X_test, y_test, min_samples_leaf in zip(X_train_array, y_train_classes, X_test_array, y_test_classes, min_samples_leafs):
    clf = RandomForestClassifier(n_estimators = 50, min_samples_leaf=min_samples_leaf, max_features=600)
    test(clf)

Training: 0.6588750160895869
Testing: 0.4779569343810593
Training: 0.9802076573653472
Testing: 0.2541047693510555
Training: 1.0
Testing: 0.16666666666666666
Training: 0.9415430267062315
Testing: 0.43221476510067114
Training: 1.0
Testing: 0.2878787878787879
Training: 0.9955837703560585
Testing: 0.3305785123966942
CPU times: user 5min 41s, sys: 1.11 s, total: 5min 42s
Wall time: 5min 42s


In [89]:
%%time
for X_train, y_train, X_test, y_test in zip(X_train_array, y_train_classes, X_test_array, y_test_classes):
    clf = SGDClassifier(loss="log", n_iter=20)
    test(clf)

Training: 0.42578195391942336
Testing: 0.4110113276864986
Training: 0.3238157040882544
Testing: 0.17982799061767005
Training: 1.0
Testing: 0.0
Training: 0.4986646884272997
Testing: 0.3825503355704698
Training: 1.0
Testing: 0.3333333333333333
Training: 0.6213083080320176
Testing: 0.2833530106257379
CPU times: user 18.6 s, sys: 590 ms, total: 19.2 s
Wall time: 18.2 s


In [105]:
%%time
for X_train, y_train, X_test, y_test in zip(X_train_array, y_train_classes, X_test_array, y_test_classes):
    clf = XGBClassifier(max_depth=10, min_child_weight=8, learning_rate=0.3, seed=0, n_estimators=100, subsample=0.80, colsample_bytree=0.80, objective="multi:softprob")
    test(clf)

Training: 0.7332861372119964
Testing: 0.4669864271864476
Training: 0.9928617780661908
Testing: 0.22830336200156373
Training: 0.22727272727272727
Testing: 0.3333333333333333
Training: 0.9899109792284867
Testing: 0.4161073825503356
Training: 0.8823529411764706
Testing: 0.16666666666666666
Training: 0.9895114545956389
Testing: 0.32585596221959856
CPU times: user 39min 48s, sys: 19.3 s, total: 40min 8s
Wall time: 11min 59s


In [117]:
%%time
min_samples_leafs = [16, 4, 1, 4, 1, 2]
for X_train, y_train, X_test, y_test, min_samples_leaf, i in zip(X_train_array, y_train_classes, X_test_array, y_test_classes, min_samples_leafs, range(6)):
    rf_clf = RandomForestClassifier(n_estimators = 50, min_samples_leaf=min_samples_leaf, max_features=600)
    sgd_clf = SGDClassifier(loss="log", n_iter=20)
    xgb_clf = XGBClassifier(max_depth=10, min_child_weight=8, learning_rate=0.3, seed=0, n_estimators=100, subsample=0.80, colsample_bytree=0.80, objective="multi:softprob")
    eclf = EnsembleVoteClassifier(clfs=[xgb_clf, rf_clf, sgd_clf], weights=[1, 1, 1], voting='soft')
    test(eclf)
    joblib.dump(eclf, 'models/note_class_eclf/clf_{0}.pkl'.format(i))

Training: 0.6738318959969108
Testing: 0.47652821716501687
Training: 0.9446787800129786
Testing: 0.21266614542611414
Training: 1.0
Testing: 0.0
Training: 0.9256676557863501
Testing: 0.43355704697986575
Training: 1.0
Testing: 0.21212121212121213
Training: 0.962186033673751
Testing: 0.31286894923258557
CPU times: user 46min 59s, sys: 29.5 s, total: 47min 29s
Wall time: 20min 1s
CPU times: user 46min 59s, sys: 29.5 s, total: 47min 29s
Wall time: 20min 1s


In [100]:
%%time
def build_model(num_classes):
    model = Sequential()

    model.add(Dense(512, input_shape=(812,)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.4))

    model.add(Dense(512))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.4))

    model.add(Dense(num_classes))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adagrad',
                  metrics=['accuracy'])
    
    return model

batch_sizes = [64, 16, 2, 16, 4, 8]
models = []
for X_train, y_train, X_test, y_test, batch_size in zip(X_train_array, y_train_array, X_test_array, y_test_array, batch_sizes):
    model = build_model(4)
    model.fit(X_train, y_train, nb_epoch=15, batch_size=batch_size, verbose=0, validation_data=(X_test, y_test))
    print ('Training: {0}'.format(model.evaluate(X_train, y_train, verbose=0)[1]))
    print ('Testing: {0}'.format(model.evaluate(X_test, y_test, verbose=0)[1]))
    models.append(model)

Training: 0.6310722100656455
Testing: 0.45096438412234935
Training: 0.5314730693967523
Testing: 0.2947615321443079
Training: 0.2727272727272727
Testing: 0.0
Training: 0.9320474777448071
Testing: 0.352348993293591
Training: 0.967914441690088
Testing: 0.3181818181818182
Training: 0.7728401876897598
Testing: 0.3069657615464018
CPU times: user 13min 40s, sys: 23.3 s, total: 14min 3s
Wall time: 7min 18s


In [9]:
np.concatenate((np.array([1, 2]), np.array([3, 4])), axis=0)

array([1, 2, 3, 4])

In [5]:
import numpy as np